# Mouse-mole-vole: The inconspicuous benefit of phonology during retrieval from semantic memory

# Importing packages

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import itertools
import scipy.spatial.distance
import pandas as pd
import numpy as np
from numpy.random import randint
from scipy.special import softmax
from sklearn.preprocessing import MinMaxScaler, normalize
from numpy.linalg import matrix_power
from functools import lru_cache
import glob
from scipy.special import expit
import matplotlib.pyplot as plt
import nltk
from functools import lru_cache
from itertools import product as iterprod
import itertools
from nltk.metrics import *



# importing norms and embeddings

Here we import the Troyer norms, as well as frequency, semantic and phonological similarity matrices. See code at the end of the notebook for computing the similarity matrices.

In [3]:
norms = pd.read_csv("data/troyernorms.csv", encoding="unicode-escape")
freq_matrix = pd.read_csv('data/animals_frequencies.csv', header = None)
labels = list(freq_matrix[0])
freq_matrix = np.array(freq_matrix[1])
sim_matrix = pd.read_csv('data/word2vec_sim_matrix.csv',header = None).values
phon_matrix = pd.read_csv('data/simlabels_phon_matrix.csv',header = None).values
print(f"size of vocab is {len(labels)} animals")

size of vocab is 771 animals


# define foraging models

## history function

In [4]:
def create_history_variables(fluency_list, labels, sim_matrix, freq_matrix, phon_matrix = None):
  '''
  inputs:
  (1) sim_matrix: semantic similarity matrix (NxN np.array)
  (2) phon_matrix: phonological similarity matrix (NxN np.array)
  (3) freq_matrix: frequencies array (Nx1 array)
  (4) labels: the space of words (list of length N)
  (5) fluency_list: items produced by a participant (list of size L)
  
  outputs:
  (1) sim_list: semantic similarities between each item in fluency_list (list of size L)
  (2) sim_history: semantic similarities of each word with all items in labels (list of L arrays of size N)
  (1) phon_list: phonological similarities between each item in fluency_list (list of size L)
  (2) phon_history: phonological similarities of each word with all items in labels (list of L arrays of size N)
  (1) freq_list: frequencies of each item in fluency_list (list of size L)
  (2) freq_history: frequencies of all items in labels repeated L items (list of L arrays of size N)
  
  
  '''
  phon_matrix[phon_matrix <= 0] = .0001
  sim_matrix[sim_matrix <= 0] = .0001

  freq_list = []
  freq_history = []

  sim_list = []
  sim_history = []

  phon_list = []
  phon_history = []

  for i in range(0,len(fluency_list)):
    word = fluency_list[i]
    currentwordindex = labels.index(word)

    freq_list.append(freq_matrix[currentwordindex])
    freq_history.append(freq_matrix)
  
    if i > 0: # get similarity between this word and preceding word
      prevwordindex = labels.index(fluency_list[i-1])
      sim_list.append(sim_matrix[prevwordindex, currentwordindex] )
      sim_history.append(sim_matrix[prevwordindex,:])
      if phon_matrix is not None:
        phon_list.append(phon_matrix[prevwordindex, currentwordindex] )
        phon_history.append(phon_matrix[prevwordindex,:])
    else: # first word
      sim_list.append(0.0001)
      sim_history.append(sim_matrix[currentwordindex,:])
      if phon_matrix is not None:
        phon_list.append(0.0001)
        phon_history.append(phon_matrix[currentwordindex,:])
  
  return sim_list, sim_history, freq_list, freq_history,phon_list, phon_history

## forage class

In [5]:
class forage:
    def model_static(beta, freql, freqh, siml, simh):
        ## beta contains the optimization parameters for frequency (beta[0]) and semantic similarity (beta[1])
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0: # if first item then its probability is based on just frequency
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else: # if not first item then its probability is based on its similarity to prev item AND frequency
            # P of item based on frequency and similarity
                numrat = pow(freql[k],beta[0]) * pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0]) * pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat) # negative Log likelihood of this item: this will be minimized eventually
        return ct
   
    def model_dynamic_psyrev(beta, freql, freqh, siml, simh, switchvals):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif switchvals[k]==1: ## "dip" based on switch type
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(freql[k],beta[0])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

    def model_static_plocal(beta, freql, freqh, siml, simh, phonl, phonh):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0: 
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else: 
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) * pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2])* pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct
      
    def model_dynamic_pswitchonly(beta, freql, freqh, siml, simh, phonl, phonh,switchvals):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif switchvals[k]==1:
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) 
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2]) )
            else:
                numrat = pow(freql[k],beta[0])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct
    

    def model_dynamic_plocal(beta, freql, freqh, siml, simh, phonl, phonh,switchvals):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif switchvals[k]==1: 
                numrat = pow(freql[k],beta[0]) 
                denrat = sum(pow(freqh[k],beta[0]))
            else:
                numrat = pow(freql[k],beta[0])*pow(phonl[k],beta[2])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(phonh[k],beta[2])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
            
        return ct

    def model_dynamic_pglobal(beta, freql, freqh, siml, simh, phonl, phonh,switchvals):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif switchvals[k]==1:
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) 
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2]) )
            else:
                numrat = pow(freql[k],beta[0])*pow(phonl[k],beta[2])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(phonh[k],beta[2])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

## switch function

In [6]:
def create_switch_lists(fluency_list, history_vars):
  '''
  takes in a fluency list and history variables and creates all possible switch/cluster designations
  current implemented methods:
  (1) simdrop
  (2) troyer norms
 
  output:
  a list of lists corresponding to 
  (1) simdrop
  (2) troyer
  '''
  simdrop = []
  troyer = []
  
  semantic_similarity = history_vars[0]
  phonological_similarity = history_vars[4]

  for k in range(len(fluency_list)):
    if (k > 0 and k < (len(fluency_list)-2)): 
      # simdrop
      if (semantic_similarity[k+1] > semantic_similarity[k]) and (semantic_similarity[k-1] > semantic_similarity[k]):
        simdrop.append(1)
      else:
        simdrop.append(0)
      
      # troyer
      item1 = fluency_list[k]
      item2 = fluency_list[k-1]
      category1 = norms[norms['Animal'] == item1]['Category'].values.tolist()
      category2 = norms[norms['Animal'] == item2]['Category'].values.tolist()
      if len(list(set(category1) & set(category2)))== 0:
          troyer.append(1)
      else:
          troyer.append(0)
    else:
        simdrop.append(2)
        troyer.append(2)
  
  finalswitches = []
  finalswitches.append(simdrop)
  finalswitches.append(troyer)

  return finalswitches 

## fmin - optimal betas

In [7]:
def fmin_output_models (modelcode, history_vars, switchvals, random_vars):
  '''
  total number of models:
  (1) psyrev static
  (2) psyrev dynamic (with any switch type)
  (3) phonology static (with any switch type)
  (4) phonology global dynamic (with any switch type)
  (5) phonology local dynamic (with any switch type)
  (6) phonology switch only dynamic (with any switch type)
  '''
  import numpy as np
  from scipy.optimize import fmin
  r1,r2,r3 = random_vars

  siml, simh,  freql, freqh, phonl, phonh = history_vars

  if modelcode == 1:
    return fmin(forage.model_static, [r1,r2,r3], args=(freql, freqh, siml, simh), ftol = 0.001, disp=False)
  elif modelcode == 2:
    return fmin(forage.model_dynamic_psyrev, [r1,r2, r3], args=(freql, freqh, siml, simh, switchvals), ftol = 0.001, disp=False)
  elif modelcode == 3:
    return fmin(forage.model_static_plocal, [r1,r2, r3], args=(freql, freqh, siml, simh, phonl, phonh), ftol = 0.001, disp=False)
  elif modelcode == 4:
    return fmin(forage.model_dynamic_pglobal, [r1,r2, r3], args=(freql, freqh, siml, simh, phonl, phonh,switchvals), ftol = 0.001, disp=False)
  elif modelcode == 5:
    return fmin(forage.model_dynamic_plocal, [r1,r2, r3], args=(freql, freqh, siml, simh, phonl, phonh, switchvals), ftol = 0.001, disp=False)
  else: 
    return fmin(forage.model_dynamic_pswitchonly, [r1,r2, r3], args=(freql, freqh, siml, simh, phonl, phonh, switchvals), ftol = 0.001, disp=False)

## run models

In [8]:
def run_model(modelcode, betas, history_vars, switchvals):
  siml, simh,  freql, freqh, phonl, phonh = history_vars

  if modelcode == 1:
    return forage.model_static(betas, freql, freqh, siml, simh)
  elif modelcode == 2:
    return forage.model_dynamic_psyrev(betas, freql, freqh, siml, simh, switchvals)
  elif modelcode == 3:
    return forage.model_static_plocal(betas, freql, freqh, siml, simh, phonl, phonh)
  elif modelcode == 4:
    return forage.model_dynamic_pglobal(betas, freql, freqh, siml, simh, phonl, phonh, switchvals)
  elif modelcode == 5:
    return forage.model_dynamic_plocal(betas, freql, freqh, siml, simh, phonl, phonh, switchvals)
  else: 
    return forage.model_dynamic_pswitchonly(betas, freql, freqh, siml, simh, phonl, phonh, switchvals)

## store data

In [9]:
def store_data(betas, nLLs, fluency_list):
  # function that formats everything correctly
  df = pd.DataFrame()
  modelnames = dict()
  modelnames[str([1,0])] = "psyrev static simdrop"
  modelnames[str([1,1])] = "psyrev static troyer"
  modelnames[str([1,2])] = "psyrev static participant"

  modelnames[str([2,0])] = "psyrev dynamic simdrop"
  modelnames[str([2,1])] = "psyrev dynamic troyer"
  modelnames[str([2,2])] = "psyrev dynamic participant"

  modelnames[str([3,0])] = "static plocal simdrop"
  modelnames[str([3,1])] = "static plocal troyer"
  modelnames[str([3,2])] = "static plocal participant"

  modelnames[str([4,0])] = "dynamic pglobal simdrop"
  modelnames[str([4,1])] = "dynamic pglobal troyer"
  modelnames[str([4,2])] = "dynamic pglobal participant"

  modelnames[str([5,0])] = "dynamic plocal simdrop"
  modelnames[str([5,1])] = "dynamic plocal troyer"
  modelnames[str([5,2])] = "dynamic plocal participant"

  modelnames[str([6,0])] = "dynamic pswitchonly simdrop"
  modelnames[str([6,1])] = "dynamic pswitchonly troyer"
  modelnames[str([6,2])] = "dynamic pswitchonly participant"
  
  df["beta_labels"] = betas.keys()
  df["beta_frequency_semantic_phonology"] = betas.values()
  df["model_names"] = df.apply(lambda x: modelnames[x['beta_labels']], axis=1)
  df["optimal_nLLs"] = nLLs[0]
  df["random_nLLs"] = nLLs[1]

  df["N"] = len(fluency_list)
  
  return df

## create fit df

In [12]:
import os

def modelFits(path, delimiter, model_list):
    ### LOAD BEHAVIORAL DATA ###
    df = pd.read_csv(path, header=None, names=['SID', 'entry', 'participantswitch'], delimiter=delimiter)

    r1 = np.random.rand()
    r2 = np.random.rand()
    r3 = np.random.rand()

    sidlist = df["SID"].unique()
    ct = 0
    allfit_df = pd.DataFrame()
    allmetrics = pd.DataFrame()
    for sid in sidlist:
      ct+=1
      print( "SUBJECT " + str(ct) + '/' + str(len(sidlist)) + " " + str(sid))
      # for each subject, run the models specified in model_list
      newdata_main = df[df["SID"] == sid]
      sub_list = list(newdata_main["entry"])
      participant_switch = list(newdata_main["participantswitch"])

      history_vars = create_history_variables(sub_list, labels, sim_matrix, freq_matrix, phon_matrix )
      switchvals = create_switch_lists(sub_list, history_vars) # contains the different switch lists

      # add participant-level switch list here
      switchvals.append(participant_switch)
      
      # obtain fmin output (betas) for different models and switch methods
      optimal_betas = {str([model_i, i]):fmin_output_models(model_i, history_vars, switchvals[i], [r1,r2,r3]).tolist() for model_i in model_list for i in range(len(switchvals))}

      optimal_fits = [run_model(model_i, optimal_betas[str([model_i,i])], history_vars, switchvals[i]) for model_i in model_list for i in range(len(switchvals))]
      random_fits = [run_model(model_i, [0]*len(optimal_betas[str([model_i,i])]), history_vars, switchvals[i]) for model_i in model_list for i in range(len(switchvals))]

      fit_df = store_data(optimal_betas, [optimal_fits, random_fits], sub_list)
      fit_df["subject"] = sid

      allfit_df = pd.concat([allfit_df, fit_df])

      # also create a df for specific switch values/history vars
      metric_df = pd.DataFrame()
      
      metric_df["item"] = sub_list
      metric_df["subject"] = sid
      metric_df["semantic"] = history_vars[0]
      metric_df["frequency"] = history_vars[2]
      metric_df["phonology"] = history_vars[4]
      metric_df["simdrop"] = switchvals[0]
      metric_df["troyer"] = switchvals[1]
      metric_df["participant"] = switchvals[2]
      
      allmetrics = pd.concat([allmetrics, metric_df])
    
    return allfit_df, allmetrics

# Run models
This cell runs the foraging models for the 171 participants (141 in HJT dataset and 30 in the LEA dataset). This process takes a few minutes (~25 minutes).

In [13]:
allfit, allmetric = modelFits('data/data-fluency.txt', delimiter = "\t", model_list=[1,2,3,4,5, 6])

SUBJECT 1/171 50001
SUBJECT 2/171 50002
SUBJECT 3/171 50003
SUBJECT 4/171 50004
SUBJECT 5/171 50006
SUBJECT 6/171 50007
SUBJECT 7/171 50008
SUBJECT 8/171 50010
SUBJECT 9/171 50011
SUBJECT 10/171 50012
SUBJECT 11/171 50013
SUBJECT 12/171 50014
SUBJECT 13/171 50016
SUBJECT 14/171 50017
SUBJECT 15/171 50018
SUBJECT 16/171 50019
SUBJECT 17/171 50020
SUBJECT 18/171 50022
SUBJECT 19/171 50023
SUBJECT 20/171 50024
SUBJECT 21/171 50025
SUBJECT 22/171 50026
SUBJECT 23/171 50027
SUBJECT 24/171 50029
SUBJECT 25/171 50030
SUBJECT 26/171 50031
SUBJECT 27/171 50032
SUBJECT 28/171 50033
SUBJECT 29/171 50034
SUBJECT 30/171 50036
SUBJECT 31/171 51
SUBJECT 32/171 71
SUBJECT 33/171 198
SUBJECT 34/171 199
SUBJECT 35/171 285
SUBJECT 36/171 286
SUBJECT 37/171 287
SUBJECT 38/171 288
SUBJECT 39/171 289
SUBJECT 40/171 290
SUBJECT 41/171 291
SUBJECT 42/171 292
SUBJECT 43/171 293
SUBJECT 44/171 294
SUBJECT 45/171 295
SUBJECT 46/171 296
SUBJECT 47/171 297
SUBJECT 48/171 298
SUBJECT 49/171 379
SUBJECT 50/171 380
S

In [14]:
display(allfit.head())
display(allmetric.head())

,beta_labels,beta_frequency_semantic_phonology,model_names,optimal_nLLs,random_nLLs,N,subject
0,"[1, 0]","[2.118849817862145, 3.3959017925169173, 2.4607...",psyrev static simdrop,161.889318,192.782963,29,50001
1,"[1, 1]","[2.118849817862145, 3.3959017925169173, 2.4607...",psyrev static troyer,161.889318,192.782963,29,50001
2,"[1, 2]","[2.118849817862145, 3.3959017925169173, 2.4607...",psyrev static participant,161.889318,192.782963,29,50001
3,"[2, 0]","[2.025309820356738, 4.01097268731335, -4.55386...",psyrev dynamic simdrop,161.455482,192.782963,29,50001
4,"[2, 1]","[2.0614933395347323, 3.635874669687796, 0.6848...",psyrev dynamic troyer,169.919394,192.782963,29,50001


,item,subject,semantic,frequency,phonology,simdrop,troyer,participant
0,horse,50001,0.000100,3.676,0.0001,2,2,2.0
1,pig,50001,0.469811,3.300,0.0001,0,0,0.0
2,bear,50001,0.297824,3.467,0.0001,1,1,1.0
3,cat,50001,0.383209,3.529,0.0001,0,1,1.0
4,dog,50001,0.760946,3.993,0.0001,0,0,0.0


In [15]:
allfit.to_csv('data/cogsci2022-fits.csv', index=False, header=True)
allmetric.to_csv('data/cogsci2022-metrics.csv', index=False, header=True)

# code for obtaining semantic and phonological similarity

Here we compute the phonemic & semantic similarity matrices for a vocabulary of 771 animals

## create semantic similarity matrix
word2vec embeddings for 771 animal words were obtained from: https://github.com/plasticityai/magnitude

In [19]:
# import embeddings
embeddings = pd.read_csv("data/animals_embeddings.csv").values
print(f"embeddings are shaped:", embeddings.shape)
print(f"vocab is {len(embeddings)} words")
len(labels)

embeddings are shaped: (771, 300)
vocab is 771 words


771

In [ ]:
def create_sim_matrix(embeddings, simlabels):
  N = len(simlabels)
  # create semantic similarity matrix
  matrix = 1-scipy.spatial.distance.cdist(embeddings, embeddings, 'cosine').reshape(-1)
  matrix = matrix.reshape((N,N))
  print("sim matrix has been created:", matrix.shape)

  w1_index = simlabels.index("dolphin")
  w2_index = simlabels.index("kitten")
  w3_index = simlabels.index("whale")

  print("dolphin-kitten:", matrix[w1_index, w2_index])
  print("dolphin-whale:", matrix[w1_index, w3_index])
  print("dolphin-dolphin:", matrix[w1_index, w1_index])

  pd.DataFrame(matrix).to_csv('data/word2vec_sim_matrix.csv', index=False, header=False)

  print("sim matrix has been saved to drive!")

create_sim_matrix(embeddings, labels)


## create phonological similarity matrix

### phoneme function

We define a phoneme extraction function, that takes an input string and returns the phonemes based on CMUDict.

In [ ]:
# algo to obtain phonemes for any given strng
# obtained from: https://stackoverflow.com/questions/33666557/get-phonemes-from-any-word-in-python-nltk-or-other-modules
try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

@lru_cache()
def wordbreak(s):
    s = s.lower()
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

def normalized_sim(w1, w2):
  return 1-edit_distance(w1,w2)/(max(len(w1), len(w2)))


In [ ]:
def create_phon_matrix(vocab):
  vocabulary = vocab.copy()
  N = len(vocabulary)
  # replace all underscores (_) with space (" ") to match with glove vectors/vocab
  import re
  vocabulary = [re.sub('[^a-zA-Z]+', '', str(v)) for v in vocabulary]
  # create phonemic similarity matrix for the small vocab
  pmatrix = np.array([normalized_sim(wordbreak(w1)[0], wordbreak(w2)[0]) for w1 in vocabulary for w2 in vocabulary]).reshape((N,N))
  print("pmatrix has been created:", pmatrix.shape)
  print(pmatrix)
  pd.DataFrame(pmatrix).to_csv('data/simlabels_phon_matrix.csv', index=False, header=False)  
  print("phon matrix has been saved to drive!")

create_phon_matrix(labels)
